In [6]:
import json
import numpy as np
import os
import pandas as pd
import urllib2

# connect to poloniex's API
url = "https://poloniex.com/public?command=returnChartData&currencyPair=USDT_BTC&start=1483238867&end=1491014867&period=300"
# parse json returned from the API to Pandas DF
openUrl = urllib2.urlopen(url)
r = openUrl.read()
openUrl.close()
d = json.loads(r.decode())
df = pd.DataFrame(d)

original_columns=[u'close', u'date', u'high', u'low', u'open']
new_columns = ['Close','Timestamp','High','Low','Open']
df = df.loc[:,original_columns]
df.columns = new_columns
df.to_csv('data/bitcoin2015to2017.csv',index=None)

In [10]:
sbtime = [['1483238867','1491014867'],['1491014867','1498877267'],['1498877267','1506826067'],['1506826067','1514774867'],['1514774867','1522550867'],['1522550867','1530413267'],['1530413267','1538362067'],['1538362067','1546310867']]
ans = []
for interval in sbtime:
    start, end = interval[0], interval[1]
    url = "https://poloniex.com/public?command=returnChartData&currencyPair=USDT_BTC&start="+start+"&end="+end+"&period=300"
    openUrl = urllib2.urlopen(url)
    r = openUrl.read()
    openUrl.close()
    d = json.loads(r.decode())
    df = pd.DataFrame(d)

    original_columns=[u'close', u'date', u'high', u'low', u'open']
    new_columns = ['Close','Timestamp','High','Low','Open']
    df = df.loc[:,original_columns]
    df.columns = new_columns
    ans.append(df)
result = pd.concat(ans)
result.to_csv('data/bitcoin2015to2017.csv',index=None)


In [15]:
result.head()

,Close,Timestamp,High,Low,Open
0,962.280022,1483238867,962.280022,962.280022,962.280022
1,962.200000,1483239000,962.200000,962.000000,962.200000
2,963.250005,1483239300,963.250005,962.000000,962.000000
3,962.000000,1483239600,962.000000,962.000000,962.000000
4,962.000000,1483239900,963.250005,962.000000,963.250005


In [11]:
import numpy as np
import pandas as pd
 
class PastSampler:
    '''
    Forms training samples for predicting future values from past value
    '''
     
    def __init__(self, N, K, sliding_window = True):
        '''
        Predict K future sample using N previous samples
        '''
        self.K = K
        self.N = N
        self.sliding_window = sliding_window
 
    def transform(self, A):
        M = self.N + self.K     #Number of samples per row (sample + target)
        #indexes
        if self.sliding_window:
            I = np.arange(M) + np.arange(A.shape[0] - M + 1).reshape(-1, 1)
        else:
            if A.shape[0]%M == 0:
                I = np.arange(M)+np.arange(0,A.shape[0],M).reshape(-1,1)
                
            else:
                I = np.arange(M)+np.arange(0,A.shape[0] -M,M).reshape(-1,1)
            
        B = A[I].reshape(-1, M * A.shape[1], A.shape[2])
        ci = self.N * A.shape[1]    #Number of features per sample
        return B[:, :ci], B[:, ci:] #Sample matrix, Target matrix

#data file path
dfp = 'data/bitcoin2015to2017.csv'

#Columns of price data to use
columns = ['Close']
df = pd.read_csv(dfp)
# df = cnm.loc[:60000,:]
time_stamps = df['Timestamp']
df = df.loc[:,columns]
original_df = pd.read_csv(dfp).loc[:,columns]


In [12]:
file_name='bitcoin2015to2017_close.h5'

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
# normalization
for c in columns:
    df[c] = scaler.fit_transform(df[c].values.reshape(-1,1))
    
#Features are input sample dimensions(channels)
A = np.array(df)[:,None,:]
original_A = np.array(original_df)[:,None,:]
time_stamps = np.array(time_stamps)[:,None,None]

#Make samples of temporal sequences of pricing data (channel)
NPS, NFS = 256, 16         #Number of past and future samples
ps = PastSampler(NPS, NFS, sliding_window=False)
B, Y = ps.transform(A)
input_times, output_times = ps.transform(time_stamps)
original_B, original_Y = ps.transform(original_A)

import h5py
with h5py.File(file_name, 'w') as f:
    f.create_dataset("inputs", data = B)
    f.create_dataset('outputs', data = Y)
    f.create_dataset("input_times", data = input_times)
    f.create_dataset('output_times', data = output_times)
    f.create_dataset("original_datas", data=np.array(original_df))
    f.create_dataset('original_inputs',data=original_B)
    f.create_dataset('original_outputs',data=original_Y)

In [14]:
import pandas as pd
import numpy as numpy
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten,Reshape
from keras.layers import Conv1D, MaxPooling1D
from keras.utils import np_utils
from keras.layers import LSTM, LeakyReLU
from keras.callbacks import CSVLogger, ModelCheckpoint
import h5py
import os
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session



os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
set_session(tf.Session(config=config))

with h5py.File(''.join(['bitcoin2015to2017_close.h5']), 'r') as hf:
    datas = hf['inputs'].value
    labels = hf['outputs'].value




step_size = datas.shape[1]
units= 50
second_units = 30
batch_size = 8
nb_features = datas.shape[2]
epochs = 100
output_size=16
output_file_name='bitcoin2015to2017_close_LSTM_1_tanh_leaky_'
#split training validation
training_size = int(0.8* datas.shape[0])
training_datas = datas[:training_size,:]
training_labels = labels[:training_size,:,0]
validation_datas = datas[training_size:,:]
validation_labels = labels[training_size:,:,0]


#build model
model = Sequential()
model.add(LSTM(units=units,activation='tanh', input_shape=(step_size,nb_features),return_sequences=False))
model.add(Dropout(0.8))
model.add(Dense(output_size))
model.add(LeakyReLU())
model.compile(loss='mse', optimizer='adam')
model.fit(training_datas, training_labels, batch_size=batch_size,validation_data=(validation_datas,validation_labels), epochs = epochs, callbacks=[CSVLogger(output_file_name+'.csv', append=True),ModelCheckpoint('weights/'+output_file_name+'-{epoch:02d}-{val_loss:.5f}.hdf5', monitor='val_loss', verbose=1,mode='min')])

W0719 23:00:27.684565 4556445120 nn_ops.py:4224] Large dropout rate: 0.8 (>0.5). In TensorFlow 2.x, dropout() uses dropout rate instead of keep_prob. Please ensure that this is intended.


Train on 617 samples, validate on 155 samples
Epoch 1/100
617/617 [==============================] - 37s 59ms/step - loss: 0.0622 - val_loss: 0.0080
Epoch 2/100
617/617 [==============================] - 35s 57ms/step - loss: 0.0273 - val_loss: 0.0011
Epoch 3/100
617/617 [==============================] - 34s 55ms/step - loss: 0.0188 - val_loss: 1.4732e-04
Epoch 4/100
617/617 [==============================] - 32s 53ms/step - loss: 0.0126 - val_loss: 5.8701e-04
Epoch 5/100
617/617 [==============================] - 32s 53ms/step - loss: 0.0096 - val_loss: 2.1445e-04
Epoch 6/100
617/617 [==============================] - 33s 53ms/step - loss: 0.0099 - val_loss: 4.9751e-04
Epoch 7/100
617/617 [==============================] - 20s 32ms/step - loss: 0.0076 - val_loss: 4.5585e-04
Epoch 8/100
617/617 [==============================] - 15s 24ms/step - loss: 0.0073 - val_loss: 4.0380e-04
Epoch 9/100
617/617 [==============================] - 22s 35ms/step - loss: 0.0057 - val_loss: 6.6569e-05

Epoch 32/100
617/617 [==============================] - 12s 20ms/step - loss: 0.0047 - val_loss: 6.9676e-05
Epoch 33/100
617/617 [==============================] - 12s 20ms/step - loss: 0.0039 - val_loss: 2.4612e-04
Epoch 34/100
617/617 [==============================] - 12s 20ms/step - loss: 0.0035 - val_loss: 3.0954e-04
Epoch 35/100
617/617 [==============================] - 12s 19ms/step - loss: 0.0043 - val_loss: 5.7253e-05
Epoch 36/100
617/617 [==============================] - 12s 19ms/step - loss: 0.0035 - val_loss: 2.9521e-05
Epoch 37/100
617/617 [==============================] - 12s 20ms/step - loss: 0.0035 - val_loss: 3.7895e-04
Epoch 38/100
617/617 [==============================] - 12s 19ms/step - loss: 0.0038 - val_loss: 1.0186e-04
Epoch 39/100
617/617 [==============================] - 12s 20ms/step - loss: 0.0038 - val_loss: 1.4253e-04
Epoch 40/100
617/617 [==============================] - 12s 19ms/step - loss: 0.0041 - val_loss: 1.3093e-04
Epoch 41/100
617/617 [======

Epoch 63/100
617/617 [==============================] - 12s 20ms/step - loss: 0.0034 - val_loss: 7.0069e-05
Epoch 64/100
617/617 [==============================] - 12s 20ms/step - loss: 0.0035 - val_loss: 1.3110e-04
Epoch 65/100
617/617 [==============================] - 12s 20ms/step - loss: 0.0037 - val_loss: 8.1420e-06
Epoch 66/100
617/617 [==============================] - 12s 20ms/step - loss: 0.0035 - val_loss: 8.1545e-05
Epoch 67/100
617/617 [==============================] - 12s 20ms/step - loss: 0.0033 - val_loss: 7.2753e-06
Epoch 68/100
617/617 [==============================] - 12s 19ms/step - loss: 0.0037 - val_loss: 2.4669e-05
Epoch 69/100
617/617 [==============================] - 12s 19ms/step - loss: 0.0038 - val_loss: 6.3411e-06
Epoch 70/100
617/617 [==============================] - 12s 20ms/step - loss: 0.0041 - val_loss: 8.1773e-05
Epoch 71/100
617/617 [==============================] - 12s 19ms/step - loss: 0.0033 - val_loss: 4.2194e-05
Epoch 72/100
617/617 [======

Epoch 94/100
617/617 [==============================] - 12s 19ms/step - loss: 0.0037 - val_loss: 6.2232e-05
Epoch 95/100
617/617 [==============================] - 12s 20ms/step - loss: 0.0045 - val_loss: 1.0592e-04
Epoch 96/100
617/617 [==============================] - 12s 20ms/step - loss: 0.0030 - val_loss: 5.3439e-06
Epoch 97/100
617/617 [==============================] - 12s 19ms/step - loss: 0.0039 - val_loss: 7.7661e-06
Epoch 98/100
617/617 [==============================] - 12s 19ms/step - loss: 0.0035 - val_loss: 7.8138e-06
Epoch 99/100
617/617 [==============================] - 12s 19ms/step - loss: 0.0036 - val_loss: 5.0078e-05
Epoch 100/100
617/617 [==============================] - 12s 20ms/step - loss: 0.0032 - val_loss: 5.1465e-05
